### tensorflow注入机制

- 使用placeholder用于占位符，
- 使用feed_dict将数据注入到占位符中

In [4]:
# tensorflow注入机制

import tensorflow as tf

a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

add = tf.add(a,b)
mul = tf.multiply(a,b)

sess = tf.InteractiveSession()

# 使用feed_dict 注入数据，由于使用了placeholder
# 将数据注入到placeholder中
print("add: %i" % sess.run(add,feed_dict={a:3,b:4}))
print("mul: %i" % sess.run(mul,feed_dict={a:3,b:4}))
print(sess.run([add,mul],feed_dict={a:3,b:4}))

add: 7
mul: 12
[7, 12]


In [4]:
# 保存模型

# 准备数据
import numpy as np
import tensorflow as tf

# 生成一个-1,1的等差数列
X_train = np.linspace(-1,1,100)
Y_train = 2*X_train + np.random.randn(*X_train.shape)*0.3

# 重置图
tf.reset_default_graph()
sess = tf.InteractiveSession()

#创建模型
#占位符:用来存放训练的数据
X = tf.placeholder("float")
Y = tf.placeholder("float")
# 模型的参数
# W生成一个随机数[-1,1],形状为一维的
W = tf.Variable(tf.random_normal([1]),name="weight") 
b = tf.Variable(tf.zeros([1]),name="bias")
# 前向传播的结构
z = tf.multiply(X,W)+b

# 反向传播计算梯度以及优化
# 定义损失函数为平方差
cost = tf.reduce_mean(tf.square(Y-z))
# 设定学习率
learning_rate = 0.01
# 选择梯度下降算法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#初始化所有的参数
init = tf.global_variables_initializer()
iter_num = 20
display_step = 2

#######保存模型的路径###########
saver = tf.train.Saver()
savedir = "models/"

sess.run(init)
plotdata = {"batchsize":[],"loss":[]}
for epoch in range(iter_num):
    for (x,y) in zip(X_train,Y_train):
        sess.run(optimizer,feed_dict = {X:x,Y:y})

    if epoch % display_step == 0:
        loss = sess.run(cost,feed_dict={X:X_train,Y:Y_train})
        print("Epoch:",epoch+1," cost=",cost," W=",sess.run(W)," b=",sess.run(b))


print("Finished!")
######保存模型########
saver.save(sess,savedir+"linearModel.cpkt")

print(" cost=",sess.run(cost,feed_dict={X:X_train,Y:Y_train})," W=",sess.run(W)," b=",sess.run(b))

d:\program files\python35\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 1  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [0.6782267]  b= [0.35785705]
Epoch: 3  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.5893728]  b= [0.11283745]
Epoch: 5  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.8363491]  b= [0.01982224]
Epoch: 7  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9004006]  b= [-0.00472769]
Epoch: 9  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9169658]  b= [-0.01108396]
Epoch: 11  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9212494]  b= [-0.01272771]
Epoch: 13  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9223565]  b= [-0.0131526]
Epoch: 15  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9226437]  b= [-0.01326276]
Epoch: 17  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9227176]  b= [-0.01329114]
Epoch: 19  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.9227365]  b= [-0.01329838]
Finished!
 cost= 0.08545048  W= [1.9227399]  b= [-0.01329968]


### 模型的保存于载入

模型的文件形式：

- checkpoint文件会记录保存信息，通过它可以定位最新保存的模型：
- .meta文件保存了当前图结构
- .index文件保存了当前参数名
- .data文件保存了当前参数值

- Meta graph:
    - 这是一个协议缓冲区(protocol buffer)，它完整地保存了Tensorflow图；即所有的变量、操作、集合等。此文件以 .meta 为拓展名。

- Checkpoint 文件：
    - 这是一个二进制文件，包含weights、biases、gradients 和其他所有变量的值。此文件以 .ckpt 为扩展名. 但是，从Tensorflow 0.11版本之后做出了一些改变。现在，不再是单一的 .ckpt 文件，而是一下两个文件：
    
    ![](imgs/1.jpg)
    
    
 #### 保存模型函数
  
 - tf.train.Saver() 创建一个保存的变量
    - var_list	Saver中存储变量集合	全局变量集合
    - reshape	加载时是否恢复变量形状	True
    - sharded 	是否将变量轮循放在所有设备上	True
    - max_to_keep	保留最近检查点个数	5
    - restore_sequentially	 是否按顺序恢复变量，模型较大时顺序恢复内存消耗 True
    - keep_checkpoint_every_n_hours 隔几个小时保存一次
    
 - saver.save()  保存模型
 - tf.train.Saver.restore() ：加载模型数据
 
 #### 引入一个pretrained模型：
 
 - 创建网络：saver = tf.train.import_meta_graph('my_test_model-1000.meta')
 
 - 加载参数： 可以通过调用tf.train.Saver()类的restore方法来加载参数
    

In [5]:
#模型的载入

# 重置图
#tf.reset_default_graph()
with tf.Session() as sess2:
    sess2.run(tf.global_variables_initializer())
    ####模型的载入，创建一个Session,然后再指出加载的文件路径####
    saver.restore(sess2,savedir+"linearModel.cpkt")
    print("x=0.2,z=",sess2.run(z,feed_dict={X:0.2}))


INFO:tensorflow:Restoring parameters from models/linearModel.cpkt
x=0.2,z= [0.3712483]


In [2]:
# 查看模型的保存内容
import tensorflow as tf

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file 

savedir = "models/"

print_tensors_in_checkpoint_file(savedir+"linearModel.cpkt",None,True)

tensor_name:  bias
[-0.01329968]
tensor_name:  weight
[1.9227399]


In [3]:
# 保存检查点

# 准备数据
import numpy as np
import tensorflow as tf

# 生成一个-1,1的等差数列
X_train = np.linspace(-1,1,100)
Y_train = 2*X_train + np.random.randn(*X_train.shape)*0.3

# 重置图
tf.reset_default_graph()
sess = tf.InteractiveSession()

#创建模型
#占位符:用来存放训练的数据
X = tf.placeholder("float")
Y = tf.placeholder("float")
# 模型的参数
# W生成一个随机数[-1,1],形状为一维的
W = tf.Variable(tf.random_normal([1]),name="weight") 
b = tf.Variable(tf.zeros([1]),name="bias")
# 前向传播的结构
z = tf.multiply(X,W)+b

# 反向传播计算梯度以及优化
# 定义损失函数为平方差
cost = tf.reduce_mean(tf.square(Y-z))
# 设定学习率
learning_rate = 0.01
# 选择梯度下降算法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#初始化所有的参数
init = tf.global_variables_initializer()
iter_num = 20
display_step = 2

#######保存模型的路径###########
saver = tf.train.Saver()
savedir = "models/"

sess.run(init)
plotdata = {"batchsize":[],"loss":[]}
for epoch in range(iter_num):
    for (x,y) in zip(X_train,Y_train):
        sess.run(optimizer,feed_dict = {X:x,Y:y})

    if epoch % display_step == 0:
        loss = sess.run(cost,feed_dict={X:X_train,Y:Y_train})
        print("Epoch:",epoch+1," cost=",cost," W=",sess.run(W)," b=",sess.run(b))
    
    
    ####在模型训练的时候就保存模型######
    # 每一epoch保存一次
    saver.save(sess,savedir+"linearmodel.cpkt",global_step=epoch)
    
    #########END##########


print("Finished!")
######保存模型########
saver.save(sess,savedir+"linearModel.cpkt")

print(" cost=",sess.run(cost,feed_dict={X:X_train,Y:Y_train})," W=",sess.run(W)," b=",sess.run(b))

Epoch: 1  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [0.9877056]  b= [0.35317853]
Epoch: 3  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.779547]  b= [0.1456653]
Epoch: 5  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [1.994774]  b= [0.06469445]
Epoch: 7  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.0506015]  b= [0.0432982]
Epoch: 9  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.0650396]  b= [0.03775795]
Epoch: 11  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.068773]  b= [0.03632535]
Epoch: 13  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.0697384]  b= [0.03595479]
Epoch: 15  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.0699878]  b= [0.03585916]
Epoch: 17  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.070053]  b= [0.03583424]
Epoch: 19  cost= Tensor("Mean:0", shape=(), dtype=float32)  W= [2.070069]  b= [0.03582808]
Finished!
 cost= 0.09457235  W= [2.0700722]  b= [0.03582692]


In [6]:
###载入保存的检查点的结果###
load_epock = 16
saver = tf.train.Saver()
savedir = "models/"

with tf.Session() as sess2:
    sess2.run(tf.global_variables_initializer())
    # 方法一：
    # saver.restore(sess2,savedir+"linearmodel.cpkt-"+str(load_epock))
    # 方法二：
    kpt = tf.train.latest_checkpoint(savedir)
    if kpt != None:
        saver.restore(sess2,kpt)
        print("x=0.2,z=",sess2.run(z,feed_dict={X:0.2}))

INFO:tensorflow:Restoring parameters from models/linearModel.cpkt
x=0.2,z= [0.44984135]


### 共享变量的机制

需要使用的函数;
- tf.get_variable(<name>, <shape>, <initializer>):
    - 通过所给的名字创建是返回一个变量
- tf.variable_scope(<scope_name>): 通过 tf.get_variable()为变量名指定命名空间.
    
 为什么需要使用共享变量的机制：
 
 我的理解是这样的，利用get_variable来代替variable，一方面可以使得参数的唯一性，另一方面，利用了共享机制后，优点像该变量成为了一个全局变量，这样就可以方便在任何地方使用，而不需要重新创建变量。
 尤其在多个模型的时候，由于其参数是相同的，只是模型有差异，这时候如果两个不同的模型需要使用同一个参数，就会体现了共享变量的优势了。
   

In [7]:
import tensorflow as tf

# 重置图
tf.reset_default_graph()

with tf.variable_scope("test1",): # 这里指定命名空间
    # 在这个命名空间下定义一个get_variable
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    
    with tf.variable_scope("test2"):
        # 在不同的命名空间下可以定义其他空间下已经命名过的名字
        var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

# 使用了reuse 为True
# 会让变量共享，而不是重新新建是个变量
with tf.variable_scope("test1",reuse=True):
    var3 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    
    with tf.variable_scope("test2"):
        var4 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
        
        
print("var1:",var1.name)
print("var2:",var2.name)
print("var3:",var3.name)
print("var4:",var4.name)

var1: test1/firstvar:0
var2: test1/test2/firstvar:0
var3: test1/firstvar:0
var4: test1/test2/firstvar:0
